# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: GPT-2
* Evaluation approach: evaluate method (Accuracy)
* Fine-tuning dataset: climatebert/climate_sentiment

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [16]:
from datasets import load_dataset


dataset = load_dataset("climatebert/climate_sentiment", split='train').train_test_split(test_size = 0.2, shuffle=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
})

In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    'gpt2', 
    num_labels = 3, 
    id2label = {0:"risk", 1:"neutral", 2:"opportunity"},
    label2id = {"risk": 0, "neutral" : 1, "opportunity": 2})

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
tokenizer.pad_token = tokenizer.eos_token

def tokenizer_function(batch):
    return tokenizer(batch['text'], padding=True, truncation= True)

train_dataset = dataset ['train'].map(tokenizer_function,batched=True)
test_dataset = dataset ['test'].map(tokenizer_function,batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [19]:
model.config.pad_token_id = model.config.eos_token_id
import torch
import numpy as np
import pandas as pd
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_metrics (eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

initial_eval_args = TrainingArguments(
    output_dir="./results/initial_evaluation",
    per_device_eval_batch_size=4
)

# Initial trainer for evaluation
initial_trainer = Trainer(
    model=model,
    args=initial_eval_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

# Perform initial evaluation
initial_eval_results = initial_trainer.evaluate()
print("Initial Evaluation Results:", initial_eval_results)

eval_results = initial_trainer.evaluate()
eval_results


training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy="epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.1,
    save_strategy="epoch"
)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Initial Evaluation Results: {'eval_loss': 2.883127212524414, 'eval_accuracy': 0.365, 'eval_runtime': 9.7373, 'eval_samples_per_second': 20.54, 'eval_steps_per_second': 5.135}


In [20]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

# CHANGED: Evaluating and printing before proceeding to training
trainer.evaluate()
eval_results = trainer.evaluate()
print(eval_results)

trainer.train()

{'eval_loss': 2.883127212524414, 'eval_accuracy': 0.365, 'eval_runtime': 10.1841, 'eval_samples_per_second': 19.639, 'eval_steps_per_second': 4.91}


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.665938,0.700000


Checkpoint destination directory ./results/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=200, training_loss=0.762962875366211, metrics={'train_runtime': 107.5088, 'train_samples_per_second': 7.441, 'train_steps_per_second': 1.86, 'total_flos': 195566052556800.0, 'train_loss': 0.762962875366211, 'epoch': 1.0})

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [22]:
from peft import LoraConfig, get_peft_model, PeftModelForSequenceClassification

peft_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1)



In [23]:
peft_model = PeftModelForSequenceClassification(model, peft_config)

peft_model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 299,520 || all params: 124,739,328 || trainable%: 0.2401167336736013


In [24]:
model.config.pad_token_id = model.config.eos_token_id

def compute_metrics (eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

peft_lora_training_args = TrainingArguments(
    output_dir = "./results/peft_result",
    evaluation_strategy="epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch"
)

In [25]:
trainer_peft = Trainer(
    model = peft_model,
    args = peft_lora_training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer_peft.train()

eval_results = trainer_peft.evaluate()
eval_results

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.630982,0.740000


Checkpoint destination directory ./results/peft_result/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.6309819221496582,
 'eval_accuracy': 0.74,
 'eval_runtime': 10.0699,
 'eval_samples_per_second': 19.861,
 'eval_steps_per_second': 4.965,
 'epoch': 1.0}

In [26]:
peft_model.save_pretrained('model/our_peft_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [27]:
from peft import AutoPeftModelForSequenceClassification
lora_new_model = AutoPeftModelForSequenceClassification.from_pretrained(
    'model/our_peft_model', 
    num_labels = 3, 
    id2label = {0:"risk", 1:"neutral", 2:"opportunity"},
    label2id = {"risk": 0, "neutral" : 1, "opportunity": 2})

lora_new_model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
perf_inference_training_args = TrainingArguments(
    output_dir = "./results/inference_result",
    evaluation_strategy="epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.1,
    save_strategy="epoch"
)

In [29]:
trainer_for_lora_new = Trainer(
    model = lora_new_model,
    args = perf_inference_training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer_for_lora_new.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.172786,0.445000
2,No log,1.026811,0.510000
3,1.333800,0.997302,0.505000
4,1.333800,0.993977,0.520000


Checkpoint destination directory ./results/inference_result/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/inference_result/checkpoint-400 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/inference_result/checkpoint-600 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/inference_result/checkpoint-800 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=800, training_loss=1.213819236755371, metrics={'train_runtime': 172.1147, 'train_samples_per_second': 18.592, 'train_steps_per_second': 4.648, 'total_flos': 784997646336000.0, 'train_loss': 1.213819236755371, 'epoch': 4.0})

In [30]:
eval_results = trainer_for_lora_new.evaluate()
eval_results

{'eval_loss': 0.9939773678779602,
 'eval_accuracy': 0.52,
 'eval_runtime': 10.1167,
 'eval_samples_per_second': 19.769,
 'eval_steps_per_second': 4.942,
 'epoch': 4.0}

The accuracy increases to 52% up from the initial 36.5% before training. The accuracy seems to increase with higher number of epochs. The validation loss also decreases with each epoch.